In [45]:
#import package
import pandas as pd

from scipy.stats import ttest_ind
from scipy.stats import f_oneway
from scipy.stats import friedmanchisquare
from scipy.stats import kruskal

import seaborn as sns

In [3]:
#cc_info = pd.read_csv('cc_info.csv')
test_table = pd.read_csv('test_table.csv')
#transactions = pd.read_csv('transactions.csv')
user_table = pd.read_csv('user_table.csv')

In [14]:
test_table.head()

,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test
0,315281,2015-12-03,Direct,Web,ES,NaN,IE,1,0
1,497851,2015-12-04,Ads,Web,ES,Google,IE,0,1
2,848402,2015-12-04,Ads,Web,ES,Facebook,Chrome,0,0
3,290051,2015-12-03,Ads,Mobile,Other,Facebook,Android_App,0,1
4,548435,2015-11-30,Ads,Web,ES,Google,FireFox,0,1


In [15]:
user_table.head()

,user_id,sex,age,country
0,765821,M,20,Mexico
1,343561,F,27,Nicaragua
2,118744,M,23,Colombia
3,987753,F,27,Venezuela
4,554597,F,20,Spain


In [16]:
all = user_table.merge(test_table,on='user_id',how='left')

In [17]:
all.head()

,user_id,sex,age,country,date,source,device,browser_language,ads_channel,browser,conversion,test
0,765821,M,20,Mexico,2015-12-02,Ads,Mobile,ES,Yahoo,Android_App,0,1
1,343561,F,27,Nicaragua,2015-12-04,Ads,Web,ES,Facebook,Safari,0,0
2,118744,M,23,Colombia,2015-11-30,Ads,Mobile,ES,Facebook,Android_App,0,1
3,987753,F,27,Venezuela,2015-12-04,SEO,Web,ES,NaN,IE,0,1
4,554597,F,20,Spain,2015-12-04,Direct,Web,ES,NaN,Chrome,0,0


In [21]:
control_group = all[all['test'] == 0]

In [20]:
exp_group = all[all['test'] == 1]

### Pre Test Varification 

In [48]:
#Age Check
stat, p = f_oneway(control_group['age'],exp_group['age'])
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('They are the same')
else:
	print('They are different')

stat=2.157, p=0.142
They are the same


In [49]:
#Device Check
stat, p = kruskal(exp_group['device'].value_counts(),control_group['device'].value_counts())
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('They are the same')
else:
	print('They are different')

stat=0.600, p=0.439
They are the same


In [50]:
#Country Check
stat, p = kruskal(exp_group['country'].value_counts(),control_group[''].value_counts())
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('They are the same')
else:
	print('They are different')

stat=0.000, p=1.000
They are the same


In [54]:
#Sex Check
stat, p = kruskal(exp_group['sex'].value_counts(),control_group['sex'].value_counts())
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('They are the same')
else:
	print('They are different')

stat=0.600, p=0.439
They are the same


In [55]:
#Source Check
stat, p = kruskal(exp_group['source'].value_counts(),control_group['source'].value_counts())
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('They are the same')
else:
	print('They are different')

stat=1.190, p=0.275
They are the same


In [56]:
#Browser_language
stat, p = kruskal(exp_group['browser_language'].value_counts(),control_group['browser_language'].value_counts())
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('They are the same')
else:
	print('They are different')

stat=0.429, p=0.513
They are the same


In [57]:
#Ads Channel
stat, p = kruskal(exp_group['ads_channel'].value_counts(),control_group['ads_channel'].value_counts())
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('They are the same')
else:
	print('They are different')

stat=0.535, p=0.465
They are the same


In [58]:
#Browser
stat, p = kruskal(exp_group['browser'].value_counts(),control_group['browser'].value_counts())
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('They are the same')
else:
	print('They are different')

stat=0.331, p=0.565
They are the same


### A/B Test Analysis

In [63]:
#Conversion
stat, p = ttest_ind(exp_group['conversion'].values,control_group['conversion'].values,equal_var=False)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('They are the same')
else:
	print('They are different')

stat=-18.291, p=0.000
They are different


It shows signicance different between control and exp group. However, it turns out that the conversion rate is even higher in the than the epxeriment group, which is weird for the a/b testing.

In [67]:
#Country Conversion rate
countries = [name for name in all['country'].unique()]

print('{0:15s} {1:>15s} {2:>15s} {3:>10s}'.format('Country', 'Test Rate', 'Control Rate', 'P-Value'))
print('-' * 65)
for country in countries:
    test_val = all[(all['country'] == country) & (all['test'] == 1)]['conversion'].values
    cont_val = all[(all['country'] == country) & (all['test'] == 0)]['conversion'].values
    
    test_mean = test_val.mean()
    cont_mean = cont_val.mean()
    p_val = ttest_ind(test_val, cont_val, equal_var=False).pvalue
    
    print('{0:15s} {1:15.5f} {2:15.5f} {3:10f}'.format(country, test_mean, cont_mean, p_val))

Country               Test Rate    Control Rate    P-Value
-----------------------------------------------------------------
Mexico                  0.05119         0.04949   0.165544
Nicaragua               0.05418         0.05265   0.780400
Colombia                0.05057         0.05209   0.423719
Venezuela               0.04898         0.05034   0.573702
Spain                       nan         0.07972        nan
Ecuador                 0.04899         0.04915   0.961512
Paraguay                0.04923         0.04849   0.883697
Bolivia                 0.04790         0.04937   0.718885


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: Mean of empty slice.
  if __name__ == '__main__':
D:\Anaconda\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Argentina               0.01373         0.01507   0.335147
Peru                    0.05060         0.04991   0.771953
Chile                   0.05130         0.04811   0.302848
Guatemala               0.04865         0.05064   0.572107
Costa Rica              0.05474         0.05226   0.687876
Honduras                0.04754         0.05091   0.471463
El Salvador             0.04795         0.05355   0.248127
Panama                  0.04937         0.04680   0.705327
Uruguay                 0.01291         0.01205   0.879764


After running deeply, it shows that each control does not have a signifcanct change on the conversion rate. 

In [79]:
exp_group['date'].value_counts()

2015-12-04    67190
2015-12-03    47383
2015-11-30    33844
2015-12-01    33840
2015-12-02    33517
Name: date, dtype: int64

The A/B testing only run through a five days, it often needs at least one weeek to pick up the weeekly pattern and also needs two weeks to run and resovle the novel effect and etc. Therefore, I would suggest to extend the testing time and although it is un-sinificance at this point, it would have a chance to show the result in the near future by simply extanding it. 